In [66]:
import re
import lightgbm as lgb
import pickle as pkl
import numpy as np
from scipy.sparse import hstack
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

In [67]:
stemmer = SnowballStemmer("english")

# Define helpers for text normalization
stopwords = {x: 1 for x in stopwords.words('english')}
non_alphanums = re.compile(u'[^A-Za-z0-9]+')

def rmsle(y, y0):
    assert len(y) == len(y0)
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(y0), 2)))

def normalize_text(text):
    return u" ".join(
        [x for x in [y for y in non_alphanums.sub(' ', text).lower().strip().split(" ")] \
         if len(x) > 1 and x not in stopwords])


In [68]:
gbm = lgb.Booster(model_file='./data/models/lgb_model_best_iter_5.txt')

print("Loading models")
pickle_model = "./data/models/wb_transform_5.pkl"
wb = pkl.load(open(pickle_model, 'rb'))

Loading models


In [69]:
train = pd.read_csv('./data/kaggle/train.csv')

In [70]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


In [110]:
author_df = pd.read_csv('./data/author_cat.csv')
author_df.head()

,author,author_cat
0,Darrell Lucus,2191
1,Daniel J. Flynn,2114
2,Consortiumnews.com,1863
3,Jessica Purkiss,4372
4,Howard Portnoy,3564


In [119]:
df_authors = author_df.drop_duplicates()

In [122]:
df_authors.to_csv('./data/authors.csv', index=False)

In [154]:
#m = df_authors.isin(['Howard Portnoy']).any()
a = df_authors[(df_authors['author'] == 'Howard Portnoy')]['author_cat']
a

4    3564
Name: author_cat, dtype: int64

In [155]:
if a.values.size > 0:
    print("yes")

yes


In [139]:
type(a.values[0])

numpy.int64

In [157]:
df_authors['author_cat'].max()+1

9805

In [159]:
train['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [100]:
d = {'stemmed_title': train['title'][1],
     'stemmed_text': train['text'][1],
     'author_cat': 1062} #1026 = 0

In [91]:
d['stemmed_title'] = ' '.join([stemmer.stem(y) for y in d['stemmed_title'].split(' ')])
d['stemmed_text'] = ' '.join([stemmer.stem(y) for y in d['stemmed_text'].split(' ')])

In [101]:
d2 = {'stemmed_title': 'Wisconsin Republicans are trying to erase their 2018 loss',
     'stemmed_text': 'Wisconsin is now expected to reduce its number of early voting days, restrict gubernatorial influence over a powerful economic agency [Democratic Gov-elect Tony] Evers sought to disband, and require legislative backing for certain decisions traditionally made by the attorney general and governor -- a move that would likely block [Democratic Attorney General-Elect John] Kaul from pulling the state out of a federal lawsuit against Obamacare. The legislature will also be able to hire its own lawyers to defend state law in court, diminishing the attorney general\'s power.',
     'author_cat': 1026} #1026 = 0

In [93]:
d2['stemmed_title'] = ' '.join([stemmer.stem(y) for y in d2['stemmed_title'].split(' ')])
d2['stemmed_text'] = ' '.join([stemmer.stem(y) for y in d2['stemmed_text'].split(' ')])

In [102]:
X_test_title = wb.transform([d['stemmed_title']])
X_test_text = wb.transform([d['stemmed_text']])
X_test_author = np.array(d['author_cat'])
X_test_author = X_test_author.reshape(-1, 1)
sparse_merge_test = hstack((X_test_title, X_test_text, X_test_author)).tocsr()

Normalize text
Extract wordbags
Normalize text
Extract wordbags


In [103]:
X_test_title2 = wb.transform([d2['stemmed_title']])
X_test_text2 = wb.transform([d2['stemmed_text']])
X_test_author2 = np.array(d2['author_cat'])
X_test_author2 = X_test_author2.reshape(-1, 1)
sparse_merge_test2 = hstack((X_test_title2, X_test_text2, X_test_author2)).tocsr()

Normalize text
Extract wordbags
Normalize text
Extract wordbags


In [104]:
pred = gbm.predict(sparse_merge_test)

In [105]:
pred

array([0.00024273])

In [106]:
pred2 = gbm.predict(sparse_merge_test2)

In [108]:
pred2[0]

0.9935573693592717